# Première étude des dataset


### *Importation des 4 fichiers csv*

In [1]:
indicators<-read.csv2("Indicators.csv", dec=".", sep=";", header=T)
economicinfo<-read.csv2("economic_info.csv", dec=".", sep=";", header=T)
datacrop<-read.csv2("MAELIA_crop_raw.csv", dec=".", sep=";", header=T)
datalivestock<-read.csv2("MAELIA_livestock_raw.csv", dec=".", sep=";", header=T)

*On renomme les colonnes pour faciliter la manipulation :*

In [2]:
date <- c("2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017")
names(indicators) <- c("Scenario", "Level", "Indicator", date)
names(economicinfo) <- c("Group", "Variable", date)
names(datacrop) <- c("Scenario", "Year", "Farm", "Parcel", "Crop", "Yield", "Area", "Revenue", "Variablecost", "Energy", "ProteinkgN", "PDIN", "ProteinN.ton", "Nitrogen", "Phosphorus", "Potassium", "Active ingredient")
names(datalivestock) <- c("Scenario", "Year", "Farm", "Milk revenue", "Feed cost")

*On réalise un premier résumé des 4 fichiers :*

In [ ]:
summary(indicators)
summary(economicinfo)
summary(datacrop)
summary(datalivestock)

### *Création d'un dataset filtrant un scénario, un niveau d'étude, une culture, et un indicateur*

*Cette fonction utilise le fichier d'output MAELIA_crop.*  
*Elle prend en argument un scénario, un niveau d'étude, une culture et un indicateur et sort un sous dataset du fichier d'origine comprenant les colonnes et les lignes choisies.*  
  
**WARNING** : bien respecter l'orthographe, les majuscules et les guillemets.    
Les différents arguments possibles sont : 
* scenario : "Baseline situation" ; "Coexistence scenario" ; "Complementarity scenario" ; "Synergetic scenario"
* level : "arable" ; "livestock" ; "territory" ; "AF1" ; "AF2" ; ... ; "AF5" ; "LF1" ; "LF2"
* culture : "BarleyS" ; "BarleyW" ; "Buckwheat" ; "FavaB" ; "Flax" ; "Fodder" ; "gMaize" ; "Gpea" ; "Hay" ; "Hemp" ; "Lucern" ; "Lupin" ; "Mix_CerG" ; "OSR" ; "sMaize" ; "Tritic" ; "WheatW"
* indicateur : "Yield" ; "Area" ; "Revenue" ; "Variablecost" ; "Energy" ; "ProteinkgN" ; "PDIN" ; "ProteinN.ton" ; "Nitrogen" ; "Phosphorus" ; "Potassium" ; "Activeingredient"


In [8]:
dataset <- function(scenario, level, culture, indicateur) {
  if (level == "arable"){
      subset1 <- subset(datacrop, Scenario == scenario)
      subset2 <- subset(subset1, Farm == "AF1" | Farm == "AF2" | Farm == "AF3" | Farm == "AF4" | Farm =="AF5")
      subset3 <- subset(subset2, Crop == culture)
      subset4 <- subset(subset3, select = c("Scenario", "Year", "Farm", "Parcel", "Crop", indicateur))
  }else if (level == "livestock"){
      subset1 <- subset(datacrop, Scenario == scenario)
      subset2 <- subset(subset1, Farm == "LF1" | Farm == "LF2")
      subset3 <- subset(subset2, Crop == culture)
      subset4 <- subset(subset3, select = c("Scenario", "Year", "Farm", "Parcel", "Crop", indicateur))
  }else if (level == "territory"){
      subset1 <- subset(datacrop, Scenario == scenario)
      subset2 <- subset(subset1, Farm == "AF1" | Farm == "AF2" | Farm == "AF3" | Farm == "AF4" | Farm =="AF5" | Farm == "LF1" | Farm == "LF2")
      subset3 <- subset(subset2, Crop == culture)
      subset4 <- subset(subset3, select = c("Scenario", "Year", "Farm", "Parcel", "Crop", indicateur))
  }else{
      subset1 <- subset(datacrop, Scenario == scenario)
      subset2 <- subset(subset1, Farm == level)
      subset3 <- subset(subset2, Crop == culture)
      subset4 <- subset(subset3, select = c("Scenario", "Year", "Farm", "Parcel", "Crop", indicateur))
  }
  return(subset4)
}

### *Création d'un dataset filtrant un scénario, une ferme, un indicateur*

In [2]:
fichierfarm <- function(scenario, farm, indicateur) {
  subset1 <- subset(datacrop, Scenario == scenario)
  subset2 <- subset(subset1, Farm == farm)
  subset3 <- subset(subset2, select = c("Year", "Crop", indicateur))
  return(subset3)
}

### *Création d'un dataset filtrant un scénario, une culture, un indicateur*

In [4]:
fichierculture <- function(scenario, crop, indicateur) {
  subset1 <- subset(datacrop, Scenario == scenario)
  subset2 <- subset(subset1, Crop == crop)
  subset3 <- subset(subset2, select = c("Year", "Farm", indicateur))
  return(subset3)
}

### *Création d'un dataset filtrant un scénario, un indicateur*

In [5]:
fichierindicateur <- function(scenario, indicateur) {
  subset1 <- subset(datacrop, Scenario == scenario)
  subset2 <- subset(subset1, select = c("Year", "Crop", "Farm", indicateur))
  return(subset2)
}

### *Création d'un dataset filtrant une ferme, un indicateur*

In [7]:
fichierscenario <- function(farm, indicateur) {
  subset1 <- subset(datacrop, Farm == farm)
  subset2 <- subset(subset1, select = c("Scenario", "Year", "Crop", "Farm", indicateur))
  return(subset2)
}

### *Fonction pour tracer l'indicateur choisi en fonction des années, pour un scénario*

*Cette fonction prend en entrée les mêmes arguments que la fonction précédente.*  
*Elle renvoie un graphique où l'indicateur choisi est tracé en fonction des années pour le scénario choisi.*  
*Si le niveau d'étude choisi est "arable", "livestock" ou "territory", les tracé pour les différentes fermes seront indiqués d'une couleur différente.*  

In [11]:
tracescenario <- function(scenario, level, culture, indicateur){
  fichier <- dataset(scenario, level, culture, indicateur)
  
  if (level == "arable"){
    
    y1 <- subset(fichier, Farm == "AF1", select = c("Year", indicateur))
    y2 <- subset(fichier, Farm == "AF2", select = c("Year", indicateur))
    y3 <- subset(fichier, Farm == "AF3", select = c("Year", indicateur))
    y4 <- subset(fichier, Farm == "AF4", select = c("Year", indicateur))
    y5 <- subset(fichier, Farm == "AF5", select = c("Year", indicateur))
    
    plot(y1$Year, y1[,2], col="red", type="l", xlab = "annees", ylab = "indicateur")
    par(new=T)
    plot(y2$Year, y2[,2], col="blue", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y3$Year, y3[,2], col="green", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y4$Year, y4[,2], col="yellow", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y5$Year, y5[,2], col="purple", type="l", xlab = "", ylab = "", axes = F)
    
    legend("bottomright",legend=c("AF1","AF2", "AF3", "AF4", "AF5"),text.col=c("red","blue", "green", "yellow", "purple"))
    
    
  }else if (level == "livestock"){

    y1 <- subset(fichier, Farm == "LF1", select = c("Year", indicateur))
    y2 <- subset(fichier, Farm == "LF2", select = c("Year", indicateur))
    
    plot(y1$Year, y1[,2], col="red", type="l", xlab = "annees", ylab = "indicateur")
    par(new=T)
    plot(y2$Year, y2[,2], col="blue", type="l", xlab = "", ylab = "", axes = F)
    
    legend("bottomright",legend=c("LF1","LF2"),text.col=c("red","blue"))
    
  }else if (level == "territory"){

    
    y1 <- subset(fichier, Farm == "AF1", select = c("Year", indicateur))
    y2 <- subset(fichier, Farm == "AF2", select = c("Year", indicateur))
    y3 <- subset(fichier, Farm == "AF3", select = c("Year", indicateur))
    y4 <- subset(fichier, Farm == "AF4", select = c("Year", indicateur))
    y5 <- subset(fichier, Farm == "AF5", select = c("Year", indicateur))
    y6 <- subset(fichier, Farm == "LF1", select = c("Year", indicateur))
    y7 <- subset(fichier, Farm == "LF2", select = c("Year", indicateur))
    
    plot(y1$Year, y1[,2], col="red", type="l", xlab = "annees", ylab = "indicateur")
    par(new=T)
    plot(y2$Year, y2[,2], col="blue", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y3$Year, y3[,2], col="green", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y4$Year, y4[,2], col="yellow", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y5$Year, y5[,2], col="purple", type="l", xlab = "", ylab = "", axes = F)
    par(new=T)
    plot(y6$Year, y6[,2], col="pink", type="l", xlab = "annees", ylab = indicateur)
    par(new=T)
    plot(y7$Year, y7[,2], col="grey", type="l", xlab = "", ylab = "", axes = F)
    
    legend("bottomright",legend=c("AF1","AF2", "AF3", "AF4", "AF5", "LF1", "LF2"),text.col=c("red","blue", "green", "yellow", "purple", "pink", "grey"))
    
  }else{
    
    y1 <- subset(fichier, Farm == level, select = c("Year", indicateur))
    plot(y1$Year, y1[,2], type="l", xlab = "annees", ylab = "indicateur")
    
  }
}